图像锐化

In [ ]:
import cv2 as cv
import numpy as np


src = cv.imread(PictureAddress)                                   # 读取图像文件
# cv.namedWindow("input", cv.WINDOW_AUTOSIZE)                     # 创建可自动调整大小的窗口
# cv.imshow("input", src)                                         # 显示图像

# 卷积核直接锐化法（3x3卷积核（Kernel））
#sharpen_op = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]], dtype=np.float32)  # 总和为1 （9 + 8*(-1) = 1）保证亮度不变
sharpen_op = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=np.float32) # 总和为1 （5 + 4*(-1) = 1）保证亮度不变
# cv.filter2D()是OpenCV中用于图像卷积运算的核心函数，通过自定义核矩阵实现多种图像处理效果
sharpen_image = cv.filter2D(src, cv.CV_32F, sharpen_op)           # 生成含负值的浮点结果
# cv.convertScaleAbs()是用于处理卷积结果的关键后处理步骤，其作用是将锐化后的浮点型数据转换为可视化的8位无符号整型图像（0-255范围）
sharpen_image = cv.convertScaleAbs(sharpen_image)

cv.imshow('input',src)                                            # 显示原图
cv.imshow('output',sharpen_image)                                 # 显示锐化后的图像

cv.waitKey(0)                                                     # 等待按键
cv.destroyAllWindows()                                            # 关闭所有窗口 

USM锐化增强算法

In [ ]:
import cv2 as cv
import numpy as np


src = cv.imread(PictureAddress)                                  # 读取图像文件
# 标准差sigma= 5、15、25
blur_img = cv.GaussianBlur(src, (0, 0), 5)                       # (0, 0)：内核大小（自动根据sigma计算）
# 非锐化掩模（通过原图(src)与高斯模糊图(blur_img)的加权差实现边缘增强）
usm = cv.addWeighted(src, 1.5, blur_img, -0.5, 0)  # 1.5和-0.5的权重组合是经典的非锐化掩模参数，最终gamma值设为0保持亮度中性

cv.imshow('input',src)                                           # 显示原图
cv.imshow('gussian',blur_img)                                    # 显示高斯模糊后的图像
cv.imshow('output',usm)                                          # 显示锐化增强后的图像


cv.waitKey(0)                                                    # 等待按键
cv.destroyAllWindows()                                           # 关闭所有窗口


傅里叶变换

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.mlab import magnitude_spectrum


img = cv.imread(PictureAddress)                                    # -1参数表示按原样加载图像（包括Alpha通道）
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)                     # BGR转化为灰度图像
# 对灰度图像的离散傅里叶变换（DFT）
# np.float32(img_gray)将灰度图像转换为32位浮点格式，这是OpenCV傅里叶变换的必要输入数据类型
dft = cv.dft(np.float32(img_gray),flags = cv.DFT_COMPLEX_OUTPUT)   # 单通道灰度图作为输入时，输出复数结果需显式指定DFT_COMPLEX_OUTPUT标志
# 执行频域中心化操作,图像频域处理
dftShift = np.fft.fftshift(dft)                                    # dft必须是通过cv2.dft()得到的复数矩阵（双通道数组）
# 执行频域数据的逆中心化操作
ishift = np.fft.ifftshift(dftShift)
# cv2.idft()执行逆傅里叶变换，将经过np.fft.fftshift()中心化处理的频域数据ishift还原为空间域图像
iImage = cv.idft(ishift)
# 计算复数矩阵幅度,应用于频域图像处理后的结果转换，与cv2.dft()和np.fft.fftshift()配合实现完整的频域处理流程
iImg = cv.magnitude(iImage[:,:,0],iImage[:,:,1])  # iImage[:,:,0]为实部（Re），iImage[:,:,1]为虚部（Im）
# 使用plt.subplot(121)和plt.subplot(122)创建1行2列的并排子图布局，左侧子图位置编号121，右侧122
plt.subplot(121),plt.imshow(img,cmap = "gray"),plt.title("original"),plt.axis('off') # plt.axis('off')隐藏坐标轴，plt.title()为每幅图添加说明性标题
plt.subplot(122),plt.imshow(iImg,cmap = "gray"),plt.title("inverse"),plt.axis('off')
plt.show()
 
cv.imshow("inverse",iImg)                                          # 显示iImg
cv.waitKey(0)                                                      # 等待按键
cv.destroyAllWindows()                                             # 关闭所有窗口

理想低通滤波

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 读取图像
img = cv2.imread(PictureAddress, 0)                               # 0 表示读取为灰度图像

# 对图像进行二维快速傅里叶变换（执行FFT及中心化）
f= np.fft.fft2(img)
fshift = np.fft.fftshift(f)                                       # 执行频域中心化操作

# 设置低通滤波器的参数
D0 = 50                                                           # 截止频率，可以根据需要调整
rows, cols = img.shape                                            # 获取输入图像的尺寸
mask = np.zeros((rows, cols), np.uint8)                           # 注意这里我们不再需要第三个维度（颜色通道）
crow, ccol = rows // 2, cols // 2                                 # 计算图像中心点坐标
for i in range(rows):                                             # 通过双重循环遍历图像每个像素位置
    for j in range(cols):
        # 计算当前点到中心的距离
        d = np.sqrt((i - crow) ** 2 + (j - ccol) ** 2)    # d = √[(i-crow)² + (j-ccol)²]
        # 应用低通滤波器 （圆形掩模）
        if d <= D0:                                               # 截止条件
            mask[i, j] = 1

# 应用滤波器
fshift_filtered = fshift * mask                                   # 实现频域成分的选择性保留/滤除
# 逆傅里叶变换
f_ishift = np.fft.ifftshift(fshift_filtered)                      # 逆中心移位 (ifftshift)
img_back = np.fft.ifft2(f_ishift)                                 # 逆傅里叶变换
img_back = np.abs(img_back)                                       # ‌取模运算，提取复数的幅度部分，获得最终可显示的真实图像数据

# 由于结果可能包含复数，我们只取幅度部分（归一化与类型转换）
# 数据归一化‌：将图像像素值线性映射到0-255范围，公式：(当前值 - 最小值)/(最大值 - 最小值)*255，确保所有像素值分布在标准灰度范围内
img_back = (img_back - np.min(img_back)) / (np.max(img_back) - np.min(img_back)) * 255
img_back = np.uint8(img_back)                                     # 将浮点型数据转换为8位无符号整型(np.uint8)
   
# 显示原图像和滤波后的图像
# 使用plt.subplot(121)和plt.subplot(122)创建1行2列的并排子图布局，左侧子图位置编号121，右侧122
plt.subplot(121), plt.imshow(img, cmap='gray'), plt.title('Original Image')
plt.xticks([]), plt.yticks([])                                    # 隐藏坐标轴刻度
plt.subplot(122), plt.imshow(img_back, cmap='gray'), plt.title('Lowpass Filtered Image')
plt.xticks([]), plt.yticks([])                                    # 隐藏坐标轴刻度
plt.show()